In [1]:
import re

import numpy as np
import pandas as pd
from tqdm import tqdm

from dankypipe import pipe

tqdm.pandas()

def isfloat(x):
    try:
        float(x)
        return True
    except:
        return False
    
def isint(x):
    try:
        int(x)
        return True
    except:
        return False

In [2]:
train = pd.read_pickle('train.pickle').sort_values(by='AvSigVersion')
test = pd.read_pickle('test.pickle').sort_values(by='AvSigVersion')

train = train.rename(columns={'HasDetections':'Target'})

cols = ['Census_OSSkuName', 'Census_OSEdition', 'MachineIdentifier']
train = train.reindex(columns=cols+['Target'])
test = test.reindex(columns=cols)

These two features express the same information but differ on occasion. I first remove any non-alphanumeric characters from both features. Next, I check to see if they match by seeing if any substring of length four exists in the other. I do it like this because many are the same but with different orderings (datacenterserver and server_datacenter for example).

I also extract the OS versions that have reduced media applications by default.
https://www.howtogeek.com/322112/what-is-an-n-or-kn-edition-of-windows/

In [3]:
def transform(df):
    df['Census_OSSkuName'] = [re.sub(r'[^a-zA-Z]+', '', str(s)) for s in df.Census_OSSkuName]
    df['Census_OSEdition'] = [re.sub(r'[^a-zA-Z]+', '', str(s)) for s in df.Census_OSEdition]

    # extract the media reduced OS versions
    OS_Reduced_Media = [
        'professionaln',
        'coren',
        'enterprisesn',
        'enterprisen',
        'professionalworkstationn',
        'cloudn',
        'educationn',
        'professionaleducationn'
    ]

    mask = [
        c[0] in OS_Reduced_Media or c[1] in OS_Reduced_Media 
        for c in df[['Census_OSSkuName', 'Census_OSEdition']].itertuples() 
    ]
    df['OS_Reduced_Media'] = mask

    for c in OS_Reduced_Media:
        df.loc[df.Census_OSSkuName == c, 'Census_OSSkuName'] = c[:-1]
        df.loc[df.Census_OSEdition == c, 'Census_OSEdition'] = c[:-1]

    # replace the obvious typo
    df.loc[
        (df.Census_OSEdition == 'enterprises') |
        (df.Census_OSSkuName == 'enterprises'),
        ['Census_OSEdition', 'Census_OSSkuName']
    ] = 'enterprise'


    # There are only one of these in the entire dataset
    df.loc[
        (df.Census_OSEdition == 'professionalsinglelanguage') |
        (df.Census_OSSkuName == 'professionalsinglelanguage'),
        ['Census_OSEdition', 'Census_OSSkuName']
    ] = 'professional'

    df.loc[
        (df.Census_OSEdition == 'professionalcountryspecific') |
        (df.Census_OSSkuName == 'professionalcountryspecific'),
        ['Census_OSEdition', 'Census_OSSkuName']
    ] = 'professional'

    df.loc[
        (df.Census_OSEdition == 'professionalcountryspecific') |
        (df.Census_OSSkuName == 'professionalcountryspecific'),
        ['Census_OSEdition', 'Census_OSSkuName']
    ] = 'professional'

    # look for substring matches
    step, subsets = 4, {}
    for s in df.Census_OSEdition.unique():
        subsets[s] = {s[i:i+step] for i in range(len(s)-step+1)}

    df['Census_OSEdSkuMatch'] = [
        any([
            x in z for x in subsets[y]
        ])
        for y, z in zip(df.Census_OSEdition, df.Census_OSSkuName)
    ]
    t = df[['Census_OSEdition', 'Census_OSSkuName', 'Census_OSEdSkuMatch', 'OS_Reduced_Media']]

    osed_props = df.Census_OSEdition.value_counts(normalize=True)
    ossku_props = df.Census_OSSkuName.value_counts(normalize=True)
    
    for ix, row in df.iloc[
        t.loc[[not b for b in t.duplicated()] & ~t.Census_OSEdSkuMatch].index][
            ['Census_OSEdition', 'Census_OSSkuName', 'Census_OSEdSkuMatch']].iterrows():
        a, b = osed_props[row.Census_OSEdition], ossku_props[row.Census_OSSkuName]
        p = b/(a+b)
        choice = np.random.binomial(1, p, 1)
        if choice == 1:
            #print(p, 1, row.Census_OSSkuName)
            df.loc[ix, 'Census_OSEdition'] = row.Census_OSSkuName
        else:
            #print(p, 0, row.Census_OSEdition)
            df.loc[ix, 'Census_OSEdition'] = row.Census_OSEdition

    df.drop(columns=['Census_OSSkuName'], inplace=True)
    return df

train = transform(train)
test = transform(test)

In [4]:
avsig = pipe.download_feature('AvSigVersion_float', cache=True)
avsig_combined = pd.concat([avsig['train'], avsig['validate']], sort=False).sort_values(by='AvSigVersion_float')

train_ = train.merge(avsig_combined, on='MachineIdentifier', how='inner', sort=False)
assert len(train_) == len(train)

train = train_
del train_

train = train.sort_values(by='AvSigVersion_float').drop(columns='AvSigVersion_float')

In [7]:
val_idx = int(len(train)*.7)
val_idx

6245038

In [8]:
cols = {'Census_OSEdition': 'Census_OSEdition_clean'}
train = train.rename(columns=cols)
test = test.rename(columns=cols)

In [5]:
train.head()

,Census_OSEdition,MachineIdentifier,Target,OS_Reduced_Media,Census_OSEdSkuMatch
0,CoreSingleLanguage,c3c4bc04dc5f1c7245a862e52634428e,0,False,False
37,CoreSingleLanguage,60031444d3ec616c6e9084be521faa04,0,False,False
38,Core,d938abff6012c1488b851247a3098160,0,False,False
39,Professional,910ddd20c6d334ca03a46d9f0008fe24,1,False,False
40,Professional,5e05d22ab9db72ccbc8e41d4bc632f64,0,False,False


In [6]:
test.head()

,Census_OSEdition,MachineIdentifier,OS_Reduced_Media,Census_OSEdSkuMatch
7252423,Professional,ec6910b4d9e0baae203e9819227659ec,False,False
6804872,Professional,ddd66992da9cbb12db76d9d874fedf8b,False,False
6882538,Professional,e05db268c5f1e48e5fa63de1f39f02d7,False,False
6856130,Professional,df81a38177efaac6b95df42ddef504e6,False,False
2544324,Professional,52eb832b198099b467d39481a77afcef,False,False


In [9]:
pbar = tqdm(total=len(train.columns)-2)

for c in train.columns:
    if c == 'MachineIdentifier' or c == 'Target':
        continue
        
    pbar.set_description(c)
    
    train_ = train[['MachineIdentifier', c]].iloc[:val_idx, :]
    val_   = train[['MachineIdentifier', c]].iloc[val_idx:, :]
    test_  =  test[['MachineIdentifier', c]]
    
    try:
        pipe.upload_feature(c, (train_, val_, test_))
    except ValueError:
        pass
    
    pbar.update(1)

Census_OSEdSkuMatch: 100%|██████████| 3/3 [01:45<00:00, 51.41s/it]    